Here, I would like to show that we can just input any phylogenetic tree, designate a time and proportion of migration, and get an msprime sim.

In [1]:
import toytree
import phymsim
import pandas as pd
import numpy as np
import msprime as ms
import tempfile
import os
import subprocess
import re
import h5py
from copy import deepcopy

In [2]:
tree = toytree.rtree.coaltree(5)
tree.draw(node_labels=True, node_sizes=15, tip_labels_align=True);
print("tree height: " + str(tree.treenode.height))

tree height: 1.0707875000000002


<svg class="toyplot-canvas-Canvas" height="275.0px" id="t62b829dce51a40bfb9b9a1d2195dd8b1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 idx: 0
name: r0
dist: 0.5130
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 0.5130
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 0.8053
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 0.1792
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 0.1792
support: 100
height: 0.0000 4 idx: 5
name: 5
dist: 0.2924
support: 100
height: 0.5130 5 idx: 6
name: 6
dist: 0.2655
support: 100
height: 0.8053 6 idx: 7
name: 7
dist: 0.8916
support: 100
height: 0.1792 7 idx: 8
name: 8
dist: 0.0001
support: 100
height: 1.0708 8

In [3]:
tree = tree.mod.node_scale_root_height(tree.treenode.height*10000*4)
print("tree height: " + str(tree.treenode.height))

tree height: 42831.50000000001


In [4]:
mod = phymsim.Model(tree,
                    Ne = 10000, 
                    mut=1e-8,
                   recomb=9e-9)

In [5]:
mod.run(10,5000,return_results=True)

In [6]:
mod.df

,locus_idx,starts,stops,bps,newicks,cumulative_bps,inferred_trees
0,0,0.000000,3692.570103,3693,"((r3:16762.69372800079873,r4:16762.69372800079...",3693,None
1,0,3692.570103,3780.573484,88,"((r3:16762.69372800079873,r4:16762.69372800079...",3781,None
2,0,3780.573484,4059.015950,278,"((r3:16762.69372800079873,r4:16762.69372800079...",4059,None
3,0,4059.015950,5000.000000,941,"((r2:41677.69540379360842,(r0:22629.5859920789...",5000,None
0,1,0.000000,257.093225,257,"((r0:30969.96626826281135,r1:30969.96626826281...",5257,None
1,1,257.093225,852.747428,596,"((r0:30969.96626826281135,r1:30969.96626826281...",5853,None
2,1,852.747428,1204.733633,352,"((r2:46236.84249649318372,(r3:23167.9471385863...",6205,None
3,1,1204.733633,1554.216388,349,"((r0:30432.64667579918387,r1:30432.64667579918...",6554,None
4,1,1554.216388,1669.878173,116,"(r0:54455.66762174513860,(r1:52253.33860588517...",6670,None
5,1,1669.878173,2061.297965,391,"((r2:46236.84249649318372,(r3:23167.9471385863...",7061,None


In [7]:
len(mod.seqs)

10

In [8]:
fs =mod.infer_trees()

In [9]:
mod.df

,locus_idx,starts,stops,bps,newicks,cumulative_bps,inferred_trees
0,0,0.000000,3692.570103,3693,"((r3:16762.69372800079873,r4:16762.69372800079...",3693,"(r0:0.0006001500,(r1:0.0000025549,r2:0.0002001..."
1,0,3692.570103,3780.573484,88,"((r3:16762.69372800079873,r4:16762.69372800079...",3781,"(r0:0.0006001500,(r1:0.0000025549,r2:0.0002001..."
2,0,3780.573484,4059.015950,278,"((r3:16762.69372800079873,r4:16762.69372800079...",4059,"(r0:0.0006001500,(r1:0.0000025549,r2:0.0002001..."
3,0,4059.015950,5000.000000,941,"((r2:41677.69540379360842,(r0:22629.5859920789...",5000,"(r0:0.0006001500,(r1:0.0000025549,r2:0.0002001..."
0,1,0.000000,257.093225,257,"((r0:30969.96626826281135,r1:30969.96626826281...",5257,"(r0:0.0002000500,(r1:0.0004001500,(r3:0.000000..."
1,1,257.093225,852.747428,596,"((r0:30969.96626826281135,r1:30969.96626826281...",5853,"(r0:0.0002000500,(r1:0.0004001500,(r3:0.000000..."
2,1,852.747428,1204.733633,352,"((r2:46236.84249649318372,(r3:23167.9471385863...",6205,"(r0:0.0002000500,(r1:0.0004001500,(r3:0.000000..."
3,1,1204.733633,1554.216388,349,"((r0:30432.64667579918387,r1:30432.64667579918...",6554,"(r0:0.0002000500,(r1:0.0004001500,(r3:0.000000..."
4,1,1554.216388,1669.878173,116,"(r0:54455.66762174513860,(r1:52253.33860588517...",6670,"(r0:0.0002000500,(r1:0.0004001500,(r3:0.000000..."
5,1,1669.878173,2061.297965,391,"((r2:46236.84249649318372,(r3:23167.9471385863...",7061,"(r0:0.0002000500,(r1:0.0004001500,(r3:0.000000..."


In [10]:
mod.df['inferred_trees'][0]

0    (r0:0.0006001500,(r1:0.0000025549,r2:0.0002001...
0    (r0:0.0002000500,(r1:0.0004001500,(r3:0.000000...
0    (r0:0.0002000500,r1:0.0002000500,(r2:0.0000024...
0    (r0:0.0002000267,(r1:0.0012009610,r2:0.0006002...
0    (r0:0.0002000500,r1:0.0002000500,((r2:0.000200...
0    (r0:0.0000000300,(r1:0.0001999700,(r2:0.002403...
0    (r0:0.0004001881,(r1:0.0002000424,r2:0.0000021...
0    (r0:0.0008004270,(r1:0.0002000267,r2:0.0008004...
0    (r0:0.0018025133,(r1:0.0010006673,r2:0.0004001...
0    (r0:0.0010008657,(r1:0.0004001067,(r3:0.000200...
Name: inferred_trees, dtype: object

In [33]:
def _call_iq(command_list):
    """ call the command as sps """
    proc = subprocess.Popen(
        command_list,
        stderr=subprocess.STDOUT, 
        stdout=subprocess.PIPE
        )
    comm = proc.communicate()
    #if proc.returncode:
    #    raise IPyradError(comm[0].decode())
    return comm[0].decode()



In [45]:
np.repeat(None,5)

array([None, None, None, None, None], dtype=object)

In [44]:
mod.df['inferred_trees'][:] = 0

KeyError: 'inferred_trees'

In [35]:
_call_iq(['iqtree','-s','newfile.fasta'])

'IQ-TREE multicore version 1.6.12 for Mac OS X 64-bit built Aug 15 2019\nDeveloped by Bui Quang Minh, Nguyen Lam Tung, Olga Chernomor,\nHeiko Schmidt, Dominik Schrempf, Michael Woodhams.\n\nHost:    Patricks-MBP-2.fios-router.home (AVX, 8 GB RAM)\nCommand: iqtree -s newfile.fasta\nSeed:    692286 (Using SPRNG - Scalable Parallel Random Number Generator)\nTime:    Wed Oct  9 01:18:38 2019\nKernel:  AVX - 1 threads (4 CPU cores detected)\n\nHINT: Use -nt option to specify number of threads because your CPU has 4 cores!\nHINT: -nt AUTO will automatically determine the best number of threads to use.\n\nReading alignment file newfile.fasta ... Fasta format detected\nAlignment most likely contains DNA/RNA sequences\nAlignment has 5 sequences with 5000 columns, 20 distinct patterns\n5 parsimony-informative, 11 singleton sites, 4984 constant sites\n    Gap/Ambiguity  Composition  p-value\n   1  r0    0.00%    passed    100.00%\n   2  r1    0.00%    passed     99.99%\n   3  r2    0.00%    passe

In [14]:
mod.namedict

{4: 'r4', 3: 'r3', 2: 'r2', 1: 'r1', 0: 'r0'}

In [13]:
np.sort(list(mod.namedict.values()))

array(['r0', 'r1', 'r2', 'r3', 'r4'], dtype='<U2')

In [ ]:
# siulate data a DF and seqarray (both stored to mod); loci can be invariant...
mod.run(...)

# alternate method for fast simulation of SNPs, makes DF and seqarray; loops to ensure nsnps=nloci
mod.run_snps(...)

# fastest simcat alternative (specialized for ensure SNPs and don't mess with DFs)
mod._run_snps(...)

In [ ]:
# look at DF 
mod.df

# look at seq arr 0
mod.seqs[0]



In [ ]:
# OPTIONAL
# write CSV to file (optinoally)
mod.df.to_csv("filname.csv")

# write phylips to dir (calls write_phy() internally?)
mod.write_seqs(path="...", concat=True)
mod.write_snps(path="...", seed=123)


In [ ]:
# infer trees for each locus 
mod.infer_trees(method="raxml")


In [ ]:
# look at results
mod.df



In [11]:
mod.seqs

{'r1': array([3, 0, 3, ..., 2, 2, 3], dtype=int8),
 'r3': array([3, 0, 3, ..., 2, 2, 3], dtype=int8),
 'r2': array([3, 0, 2, ..., 2, 2, 3], dtype=int8),
 'r0': array([3, 0, 2, ..., 2, 2, 3], dtype=int8),
 'r4': array([3, 0, 2, ..., 2, 2, 3], dtype=int8)}

In [ ]:
mod.write_phy(loc=0, path=...)

### goals
1. restructure Class object for API mode
2. complete write_seqs, write_phy, and infert_trees methods.
3. Run example code (init object, .run(), .infer_tree(), .df.to_csv()) for a couple data sets (trees, Nes, admixture) for simcat
4. write data for simcat to analyze...
5. Test simcat on getting results correct

In [45]:
seqs = np.random.randint(0,4,size=(10,5,500))
seqs[0]

array([[3, 3, 0, ..., 2, 1, 2],
       [0, 1, 1, ..., 0, 2, 1],
       [3, 2, 1, ..., 0, 1, 1],
       [3, 1, 3, ..., 0, 2, 2],
       [1, 2, 3, ..., 3, 3, 3]])

In [34]:
newsim= next(mod._get_locus_sim(40000))

In [38]:
df, seqs = mod.run_several_loci(5,5000,return_results=True)#,outfile='testseveralplease4.hdf5',return_results=False)

25000

In [14]:
mod.run_locus(500,'tableout4.hdf5',force=True)

In [6]:
testit = h5py.File('testseveralplease4.hdf5','r')

In [33]:
np.array(testit['0']['seqs']['r0'])

array([3, 3, 0, 3, 2, 1, 2, 1, 1, 1, 3, 2, 3, 1, 2, 1, 0, 2, 1, 2, 1, 3,
       2, 3, 2, 1, 2, 2, 0, 0, 3, 2, 1, 0, 3, 3, 1, 1, 1, 2, 2, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 2, 3, 2, 1, 2, 2, 0, 2, 1,
       1, 1, 2, 3, 3, 1, 1, 0, 1, 0, 0, 2, 2, 0, 0, 2, 0, 2, 1, 2, 0, 1,
       0, 1, 3, 0, 2, 1, 3, 2, 1, 0, 3, 0, 1, 2, 1, 1, 3, 0, 3, 1, 1, 0,
       3, 1, 1, 3, 2, 0, 3, 0, 2, 1, 0, 0, 0, 2, 3, 0, 2, 2, 1, 1, 0, 2,
       1, 0, 0, 1, 3, 1, 0, 3, 3, 3, 1, 2, 2, 3, 2, 2, 1, 1, 0, 3, 2, 0,
       0, 3, 1, 3, 1, 0, 0, 3, 0, 2, 2, 1, 1, 3, 0, 2, 2, 1, 3, 0, 0, 3,
       0, 1, 3, 3, 3, 1, 2, 0, 0, 1, 0, 1, 1, 3, 0, 2, 0, 2, 1, 3, 2, 2,
       1, 1, 1, 2, 2, 3, 1, 0, 0, 1, 2, 1, 2, 3, 3, 1, 3, 2, 3, 3, 1, 3,
       1, 3, 2, 1, 2, 3, 1, 3, 3, 3, 3, 2, 1, 0, 1, 1, 0, 0, 3, 1, 3, 3,
       1, 1, 3, 0, 2, 1, 1, 1, 2, 3, 3, 2, 0, 3, 1, 3, 1, 2, 3, 0, 3, 3,
       3, 0, 1, 1, 1, 3, 1, 0, 1, 0, 1, 1, 1, 3, 3, 2, 1, 3, 0, 0, 2, 1,
       3, 3, 2, 3, 3, 2, 3, 2, 2, 3, 2, 2, 0, 0, 0,

In [ ]:
# __getitem__ and __setitem__ for user querying the hdf5 -- looking for index zero but not jumping through 
# acting like the __dict__ function for every class...
# could make a seqs object that's separate
# mostly want user to not have to deal with a mess

# keep sequences mostly under the hood -- for use by e.g. RAxML or mrbayes.
# for tree inferences we want to write trees out as they're inferred...
# include functions for writing out phylip

# we want to initialize object -- sets up demographic scenario / msprime simulation.
# maybe separate out the 


In [35]:
testit['0']['seqs']

<HDF5 group "/0/seqs" (5 members)>

In [14]:
list(testit['0']['df']['table'].attrs.items())

[('CLASS', b'TABLE'),
 ('FIELD_0_FILL', 0),
 ('FIELD_0_NAME', b'index'),
 ('FIELD_1_FILL', 0),
 ('FIELD_1_NAME', b'locus_idx'),
 ('FIELD_2_FILL', 0),
 ('FIELD_2_NAME', b'starts'),
 ('FIELD_3_FILL', 0.0),
 ('FIELD_3_NAME', b'stops'),
 ('FIELD_4_FILL', 0),
 ('FIELD_4_NAME', b'bps'),
 ('FIELD_5_FILL', b''),
 ('FIELD_5_NAME', b'newicks'),
 ('NROWS', 1),
 ('TITLE', Empty(dtype=dtype('S1'))),
 ('VERSION', b'2.7'),
 ('bps_dtype', b'int64'),
 ('bps_kind', b'(lp0\nVbps\np1\na.'),
 ('bps_meta', b'N.'),
 ('index_kind', b'integer'),
 ('locus_idx_dtype', b'int64'),
 ('locus_idx_kind', b'(lp0\nVlocus_idx\np1\na.'),
 ('locus_idx_meta', b'N.'),
 ('newicks_dtype', b'bytes1520'),
 ('newicks_kind', b'(lp0\nVnewicks\np1\na.'),
 ('newicks_meta', b'N.'),
 ('starts_dtype', b'int64'),
 ('starts_kind', b'(lp0\nVstarts\np1\na.'),
 ('starts_meta', b'N.'),
 ('stops_dtype', b'float64'),
 ('stops_kind', b'(lp0\nVstops\np1\na.'),
 ('stops_meta', b'N.')]

In [36]:
pd.read_hdf('testseveralplease4.hdf5','0/df')

,locus_idx,starts,stops,bps,newicks
0,0,0,500.0,500,"(r0:36428.70987931636773,((r2:13957.4741410173..."


In [25]:
list(testit['df']['table'].attrs.items())

[('CLASS', b'TABLE'),
 ('FIELD_0_FILL', 0),
 ('FIELD_0_NAME', b'index'),
 ('FIELD_1_FILL', 0),
 ('FIELD_1_NAME', b'locus_idx'),
 ('FIELD_2_FILL', 0.0),
 ('FIELD_2_NAME', b'starts'),
 ('FIELD_3_FILL', 0.0),
 ('FIELD_3_NAME', b'stops'),
 ('FIELD_4_FILL', 0),
 ('FIELD_4_NAME', b'bps'),
 ('FIELD_5_FILL', b''),
 ('FIELD_5_NAME', b'newicks'),
 ('NROWS', 2),
 ('TITLE', Empty(dtype=dtype('S1'))),
 ('VERSION', b'2.7'),
 ('bps_dtype', b'int64'),
 ('bps_kind', b'(lp0\nVbps\np1\na.'),
 ('bps_meta', b'N.'),
 ('index_kind', b'integer'),
 ('locus_idx_dtype', b'int64'),
 ('locus_idx_kind', b'(lp0\nVlocus_idx\np1\na.'),
 ('locus_idx_meta', b'N.'),
 ('newicks_dtype', b'bytes1504'),
 ('newicks_kind', b'(lp0\nVnewicks\np1\na.'),
 ('newicks_meta', b'N.'),
 ('starts_dtype', b'float64'),
 ('starts_kind', b'(lp0\nVstarts\np1\na.'),
 ('starts_meta', b'N.'),
 ('stops_dtype', b'float64'),
 ('stops_kind', b'(lp0\nVstops\np1\na.'),
 ('stops_meta', b'N.')]

In [13]:
please=pd.read_hdf('testseveralplease3.hdf5', '0/df')

NotImplementedError: Support for generic buffers has not been implemented.

In [12]:
please

,locus_idx,starts,stops,bps,newicks
0,0,0.000000,125.138745,125,"(r4:74858.76394790643826,(r3:21927.98150527140..."
1,0,125.138745,230.212692,105,"(r4:67665.37517826807743,(r3:21927.98150527140..."
2,0,230.212692,499.156445,269,"((r3:12353.48029306749413,r4:12353.48029306749..."
3,0,499.156445,500.000000,1,"((r3:12353.48029306749413,r4:12353.48029306749..."


In [42]:
please['newicks'][1]

'(r2:30883.13418628916043,((r3:7639.33577175238497,r4:7639.33577175238497):18651.84827182642766,(r0:20476.09021465267870,r1:20476.09021465267870):5815.09382892613212):4591.95014271034961);'

In [32]:
tab = testit['df']['table']

In [ ]:
tab.read_direct()

In [17]:
list(testit['df']['axis1'])

[0, 1]

In [19]:
list(testit['df'])

['axis0',
 'axis1',
 'block0_items',
 'block0_values',
 'block1_items',
 'block1_values',
 'block2_items',
 'block2_values']

In [ ]:
ID = sa2['ID']
sa2 = nprec.drop_fields(sa2, 'ID')
Make data frame with index ID using sa2

df2 = pd.DataFrame(sa2, index=ID)
df2.index.name = 'ID'

In [20]:
df['cumulative_bases'] = [9,4,5,6,2,1,6,3,4]

In [21]:
df

,locus_idx,starts,stops,bps,newicks,cumulative_bases
0,0,0.000000,500.000000,500,"(r4:65073.76135861045623,(r2:22288.07209097589...",9
0,1,0.000000,419.536504,420,"(r4:43433.02034773760533,(r3:25719.18091019288...",4
1,1,419.536504,500.000000,80,"(r4:43433.02034773760533,(r3:25719.18091019288...",5
0,2,0.000000,388.329501,388,"((r3:12749.75788796603774,(r1:12379.4195854154...",6
1,2,388.329501,474.313200,86,"((r3:12749.75788796603774,(r1:12379.4195854154...",2
2,2,474.313200,500.000000,26,"((r3:12749.75788796603774,(r1:12379.4195854154...",1
0,3,0.000000,200.641092,201,"((r0:9767.83164165844391,r2:9767.8316416584439...",6
1,3,200.641092,348.781676,148,"((r0:9767.83164165844391,r2:9767.8316416584439...",3
2,3,348.781676,500.000000,151,"((r0:9767.83164165844391,r2:9767.8316416584439...",4


In [6]:
cumulative_bps = 0
cumulative_list = []
for i in df['bps']:
    cumulative_bps+=i
    cumulative_list.append(cumulative_bps)

NameError: name 'df' is not defined

In [19]:
cumulative_list

[0, 500, 920, 1000, 1388, 1474, 1500, 1701, 1849, 2000]

In [7]:
tst = h5py.File('sevtest2.hdf5')

In [15]:
len(list(tst['0']['seqs']['r0']))

500

In [5]:
mod._get_demography()

[{'type': 'mass_migration', 'time': 2528, 'source': 1, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 2528, 'growth_rate': None, 'initial_size': 10000, 'population': 0},
 {'type': 'mass_migration', 'time': 5033, 'source': 2, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 5033, 'growth_rate': None, 'initial_size': 10000, 'population': 0},
 {'type': 'mass_migration', 'time': 13772, 'source': 4, 'dest': 3, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 13772, 'growth_rate': None, 'initial_size': 10000, 'population': 3},
 {'type': 'mass_migration', 'time': 27446, 'source': 3, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 27446, 'growth_rate': None, 'initial_size': 10000, 'population': 0}]

In [9]:
mod.run_locus(50000,outfile="testingnotebook2.hdf5",seqgen=True,force=False)

ValueError: The designated outfile named already exists. Use `force=True` to overwrite.

In [12]:
db=h5py.File('testingnotebook2.hdf5')

In [18]:
np.array(db['seqs']['r0'])

array([0, 1, 3, ..., 3, 3, 1], dtype=int8)

In [65]:
gt_df

,starts,stops,bps,newicks
0,0.000000,9794.208573,9794,"(r2:35812.80925018029666,(r4:19347.12309009667..."
1,9794.208573,36426.647144,26633,"(r2:59960.95769239007495,(r4:19347.12309009667..."
2,36426.647144,37798.936814,1372,"((r4:19347.12309009667661,(r0:3372.37519198724..."
3,37798.936814,46331.641403,8533,"((r4:19347.12309009667661,(r0:3372.37519198724..."
4,46331.641403,50000.000000,3668,"((r1:35812.80925018029666,(r2:11632.2892871420..."


In [73]:
toytree.tree(gt_df['newicks'][1]).draw()

(<toyplot.canvas.Canvas at 0x11fa515c0>,
 <toyplot.coordinates.Cartesian at 0x11fa51ef0>)

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t088fb2c35aa04838b40e7dd6d85a152d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r3 r0 r1 r4 r2

In [74]:
seqs

{'r2': array([3, 1, 0, ..., 2, 0, 0], dtype=int8),
 'r4': array([3, 1, 0, ..., 2, 0, 0], dtype=int8),
 'r1': array([3, 1, 0, ..., 2, 0, 0], dtype=int8),
 'r0': array([3, 1, 0, ..., 2, 0, 0], dtype=int8),
 'r3': array([3, 1, 0, ..., 2, 0, 0], dtype=int8)}

In [ ]:
mod = phymsim.Model(tree,
                    Ne = 10000, 
                    mut=1e-8, 
                    admixture_edges=(0, #source
                                     3, #dest
                                     800, # time
                                     .1)) # proportion

## Now make the node values correspond to thetas:

In [75]:
idx = 0
Nes=[10000,20000,30000,40000,50000,60000,70000,80000,90000]
for node in tree.treenode.traverse():
    node.add_feature('Ne', Nes[idx])
    idx+=1

tree height: 4145.004


<svg class="toyplot-canvas-Canvas" height="275.0px" id="t937f7c47fe274bf28a11f781f9ce58b7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 255.0 275.0" width="255.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 idx: 0
name: 0
dist: 1386.2760
support: 100
height: 0.0000
Ne: 90000 0 idx: 1
name: 1
dist: 1386.2760
support: 100
height: 0.0000
Ne: 80000 1 idx: 2
name: 2
dist: 2095.9880
support: 100
height: 0.0000
Ne: 60000 2 idx: 3
name: 3
dist: 2889.2440
support: 100
height: 0.0000
Ne: 40000 3 idx: 4
name: 4
dist: 4145.0040
support: 100
height: 0.0000
Ne: 20000 4 idx: 5
name: 5
dist: 709.7120
support: 100
height: 1386.2760
Ne: 70000 5 idx: 6
name: 6
dist: 793.2560
support: 100
height: 2095.9880
Ne: 50000 6 idx: 7
name: 7
dist: 1255.7600
support: 100
height: 2889.2440
Ne: 30000 7 idx: 8
name: 8
dist: 2.0000
support: 100
height: 4145.0040
Ne: 10000 8

In [76]:
tree.draw(node_labels=tree.get_node_values("Ne", True, True), node_sizes=30, tip_labels_align=True);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t1d9e7e9c88af48cdb46e4552a736aa47" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 90000 80000 60000 40000 20000 70000 50000 30000 10000

In [77]:
norm_thetas = list(np.array(tree.get_edge_values("Ne"))/10000)
tree.draw(edge_widths=norm_thetas,tip_labels=True,);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="tb084f7eac14b4635babc38026ae6c7d3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 260.0 275.0" width="260.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4

In [78]:
mod = phymsim.Model(tree,
                    Ne = None, 
                    mut=1e-8,
                   recomb=1e-9)

In [79]:
mod._get_demography()

[{'type': 'mass_migration', 'time': 1386, 'source': 1, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 1386, 'growth_rate': None, 'initial_size': 70000, 'population': 0},
 {'type': 'mass_migration', 'time': 2095, 'source': 2, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 2095, 'growth_rate': None, 'initial_size': 50000, 'population': 0},
 {'type': 'mass_migration', 'time': 2889, 'source': 3, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 2889, 'growth_rate': None, 'initial_size': 30000, 'population': 0},
 {'type': 'mass_migration', 'time': 4145, 'source': 4, 'dest': 0, 'proportion': 1.0},
 {'type': 'population_parameters_change', 'time': 4145, 'growth_rate': None, 'initial_size': 10000, 'population': 0}]

In [80]:
[i.initial_size for i in mod._get_popconfig()] # this tells us the size of each population index

[90000, 80000, 60000, 40000, 20000]

In [81]:
gt_df2, seqs2 = mod.run_locus(50000,seqgen=True)
gt_df2

,starts,stops,bps,newicks
0,0.000000,19455.908127,19456,"(r0:17304.66686415578806,((r1:6991.97506137920..."
1,19455.908127,24868.687562,5413,"((r1:6991.97506137920209,r4:6991.9750613792020..."
2,24868.687562,27339.247434,2470,"((r1:6991.97506137920209,r4:6991.9750613792020..."
3,27339.247434,31949.423471,4610,"((r1:6991.97506137920209,r4:6991.9750613792020..."
4,31949.423471,34001.191550,2052,"((r1:6991.97506137920209,r4:6991.9750613792020..."
5,34001.191550,43708.421009,9707,"((r1:6991.97506137920209,r4:6991.9750613792020..."
6,43708.421009,47286.784474,3579,"((r1:6991.97506137920209,r4:6991.9750613792020..."
7,47286.784474,50000.000000,2713,"((r1:6991.97506137920209,r4:6991.9750613792020..."


In [89]:
(np.array(list(range(5,20)))-3)/2

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5, 7. ,
       7.5, 8. ])

In [91]:
np.array(gt_df2)

array([[0.0, 19455.908127002396, 19456,
        '(r0:17304.66686415578806,((r1:6991.97506137920209,r4:6991.97506137920209):9186.02880066223588,(r2:7070.72458629240009,r3:7070.72458629240009):9107.27927574903879):1126.66300211435009);'],
       [19455.908127002396, 24868.687562381077, 5413,
        '((r1:6991.97506137920209,r4:6991.97506137920209):31581.69362858660679,(r0:17304.66686415578806,(r2:7070.72458629240009,r3:7070.72458629240009):10233.94227786338888):21269.00182581002082);'],
       [24868.687562381077, 27339.24743424618, 2470,
        '((r1:6991.97506137920209,r4:6991.97506137920209):31581.69362858660679,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):31502.94410367340970);'],
       [27339.24743424618, 31949.423470522608, 4610,
        '((r1:6991.97506137920209,r4:6991.97506137920209):76064.11481685213221,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):75985.36529193892784);'],
       [3

In [93]:
gt_df2['starts']

0        0.000000
1    19455.908127
2    24868.687562
3    27339.247434
4    31949.423471
5    34001.191550
6    43708.421009
7    47286.784474
Name: starts, dtype: float64

In [94]:
import h5py

In [149]:
here= h5py.File("./testit2.hdf5")

In [150]:
seqs = here.create_group('seqs')

In [153]:
for i in seqs2.keys():
    print(i)

r0
r1
r4
r2
r3


In [154]:
for i in seqs2.keys():
    seqs.create_dataset(i,data=seqs2[i])

In [99]:
writeit.create_dataset('starts',shape=(len(gt_df2['starts']),),data = gt_df2['starts'])

<HDF5 dataset "starts": shape (8,), type "<f8">

In [115]:
dt = h5py.special_dtype(vlen=str)

dset = writeit.create_dataset('newicks1', (len(gt_df2['newicks']),), dtype=dt)

In [116]:
dset = gt_df2['newicks']

In [110]:
toytree.tree(dset[0])

In [123]:
writeit['newicks3'] = gt_df2['newicks'].astype('S')

In [125]:
list(writeit['newicks3'])

[b'(r0:17304.66686415578806,((r1:6991.97506137920209,r4:6991.97506137920209):9186.02880066223588,(r2:7070.72458629240009,r3:7070.72458629240009):9107.27927574903879):1126.66300211435009);',
 b'((r1:6991.97506137920209,r4:6991.97506137920209):31581.69362858660679,(r0:17304.66686415578806,(r2:7070.72458629240009,r3:7070.72458629240009):10233.94227786338888):21269.00182581002082);',
 b'((r1:6991.97506137920209,r4:6991.97506137920209):31581.69362858660679,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):31502.94410367340970);',
 b'((r1:6991.97506137920209,r4:6991.97506137920209):76064.11481685213221,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):75985.36529193892784);',
 b'((r1:6991.97506137920209,r4:6991.97506137920209):95115.58412385180418,(r3:7070.72458629240009,(r0:6768.01376796623390,r2:6768.01376796623390):302.71081832616619):95036.83459893859981);',
 b'((r1:6991.97506137920209,r4:6991.97506137920